## 모듈 불러오기

In [ ]:
#delete cell
import pandas as pd

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from collections import Counter
from collections import defaultdict

import re

from tqdm import tqdm


## pip install 
- 필요시 마크다운 해제 후 사용

## Option
- 1. 출력 셀 전체 보기

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

- 2. 출력 셀 전체 보기 해제

In [ ]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

## 파일 불러오기
- 1. 'cs.' 으로 시작하는 카테고리 (arxiv data)

In [ ]:
docs1_path= 'vscode/arxiv_cs.csv'
docs1 = pd.read_csv(docs1_path)

In [ ]:
docs1.info()

In [ ]:
docs2_path = 'vscode/paperswithcode.csv'
docs2 = pd.read_csv(docs2_path)

In [ ]:
docs3_path = 'vscode/modified.csv'
docs3 = pd.read_csv(docs3_path)

- Test Dataset

In [ ]:
docs4 = docs1.head(10000)
docs4.to_csv('vscode/test_arxiv_cs.csv', index=False)

In [ ]:
docs4.info()

In [ ]:
docs5_path = 'vscode/modified.csv'
docs5 = pd.read_csv(docs5_path)

In [ ]:
docs5.info()

In [ ]:
true_count = docs5[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least2 = len(docs5[true_count >= 3])
true_count_least2

#rf'\b\d*{process_model_name(tmp_model)}s?\d*\b' -> 2908 257 10 

## Main

### 목적
- 1) title & abstract의 키워드를 통해 categories 예측
    - 방법1) tokenizing -> 각 카테고리별 빈도수 높은 단어 추출
    - 방법2) paperswithcode의 카테고리별 모델명 불러오기
    

### 1. 데이터 분석

In [ ]:
docs1. head(3)

In [ ]:
docs1.isnull().sum()
# 결측치 다수 : comments, journal-ref, doi, report-no 

In [ ]:
docs1.info()

### 2. tokenizing

1) cs.AI

In [ ]:
cs_ai_docs = docs1[docs1['categories'].str.contains('cs.AI')]

In [ ]:
cs_ai_docs.head(3)

In [ ]:
cs_ai_abstract = cs_ai_docs.abstract

In [ ]:
# tokenize를 통한 abstract의 corpus 추출
tokenized_cs_ai_abstract = [nltk.word_tokenize(i) for i in cs_ai_abstract]
tokenized_cs_ai_abstract

In [ ]:
filtered_tokens = []
stop_words = set(stopwords.words('english')) # 불용어 제거

for abstract_tokens in tokenized_cs_ai_abstract:
    # isalnum() : 숫자&문자 
    # 불용어가 아니고 숫자&문자로 이뤄진 단어만 추출
    cleaned_tokens = [token.lower() for token in abstract_tokens if token.isalnum() and token.lower() not in stop_words]
    filtered_tokens.extend(cleaned_tokens)

# 단어별 빈도수 계산
counter = Counter(filtered_tokens)
most_common_words = counter.most_common(30)

In [ ]:
most_common_words

In [ ]:
# 결론 
# tokenize로 유의미한 결과를 도출할 수 없다.
# -> 해당 결과로 category를 분류하기엔 무리가 있다고 판단

### 3. 모델에 따른 카테고리화 ex. VGG

1) cs.AI

In [ ]:
# abstract에서 VGG(대소문자 구분X)를 포함하는 행 찾기
filtered_cs_ai_docs = cs_ai_docs[cs_ai_docs['abstract'].str.contains(re.compile("VGG", re.I))]
display(len(cs_ai_docs))
display(len(filtered_cs_ai_docs))

In [ ]:
tmp = filtered_cs_ai_docs.loc[:,"abstract"]
tmp.iloc[0]
# 해당 구문에 VGGNET 이 들어가 있음

2) cs.CV

In [ ]:
cs_cv_docs = docs1[docs1['categories'].str.contains('cs.CV')]
filtered_cs_cv_docs = cs_cv_docs[cs_cv_docs['abstract'].str.contains(re.compile("VGG", re.I))]
len(filtered_cs_cv_docs)

3) cs.DM (Discrete Mathmatics)

In [ ]:
cs_dm_docs = docs1[docs1['categories'].str.contains('cs.DM')]
filtered_cs_dm_docs = cs_dm_docs[cs_dm_docs['abstract'].str.contains(re.compile("VGG", re.I))]
len(filtered_cs_dm_docs)

In [ ]:
# 결론 
# tokenize(방법1)에 비해 유의미한 결론을 도출할 수 있었음
# 대분류, 즉 CV와 NLP에서 사용하는 모델은 차이가 있기에 분류 기준으로서 적합하다고 판단
# paperswithcode의 카테고리별 모델을 불러와서 위의 코드를 실행해도 좋을 듯

### 4. 모델에 따른 카테고리화 ex. VGG, ResNet, AlexNet

1) cs.AI

In [ ]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_ai_docs[cs_ai_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

2) cs.CV

In [ ]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_cv_docs[cs_cv_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

3) cs.DM

In [ ]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_dm_docs[cs_dm_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

### 5. paperswithcode dataset (크롤링)

In [ ]:

'''
1. see all categories (click) 
    기존 : (6,3) : general 제외, graph 예외처리 
    변경 : (4,3)클릭 위치 변경을 통한 예외를 없애줌 
<기존>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(6) > a    
    
<변경>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(4) > div > h4 > a
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(7) > div > h4 > a


2. method (고정값)
body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1


3. see all models (click)
    (3,3)

body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child(3) > a

4. categories (고정값)
body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1

5. models
    (1,1) : category에 해당하는 모델이 없는 경우(예외 처리)
    #methodsTable > tbody > tr:nth-child(1) > td:nth-child(1) > div.method-image > a
    #methodsTable > tbody > tr:nth-child(2) > td:nth-child(1) > div.method-image > a
'''

- Main Code (주석 해제 후 사용)

In [ ]:
'''
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()
driver.get('https://paperswithcode.com/methods')
data_list = []  
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tmp1=4
while True: # 1,2번   
    try:
        click1_selector = f'body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child({tmp1}) > div > h4 > a'
        temp_categories = driver.find_element(By.CSS_SELECTOR,click1_selector) #
    except NoSuchElementException:       
            break
    driver.execute_script("arguments[0].click();", temp_categories)
    tmp1 +=3
    time.sleep(1.5)
    
    # 클릭으로 인한 페이지 업데이트
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    method_selector = 'body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1'
    method_name = soup.select(method_selector)
    if not method_name:
        break
    method = method_name[0].text.strip().replace('\n','')
    print('method: ',method)
    
    tmp2 = 3
    while True: # 3,4번
        try:
            click2_selector = f'body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child({tmp2}) > a'
            temp_model = driver.find_element(By.CSS_SELECTOR,click2_selector)
        except NoSuchElementException: 
            break
        driver.execute_script("arguments[0].click();", temp_model)
        tmp2 += 3
        time.sleep(1.5)
        
        # 페이지 업데이트
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #here
        category_selector = 'body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1'
        category_name = soup.select(category_selector)
        if not category_name:
            break
        category = category_name[0].text.split("\xa0")[0].strip()
        print('category: ',category)

        model_list = []
        tmp3 = 1
        while True: # 5번
            model_selector = f'#methodsTable > tbody > tr:nth-child({tmp3}) > td:nth-child(1) > div.method-image > a'
            model_name = soup.select(model_selector) 
            if not model_name:
                break
            model = model_name[0].text.strip().replace('\n', '')
            model_list.append(model)
            print("model: ",model)
            tmp3 += 1
            
        data_list.append([method,category,', '.join(model_list)])
        
        driver.back()
    driver.back()

# 데이터를 CSV 파일로 저장
with open('cw_paperswithcode.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['method', 'category', 'model'])  # 헤더 추가
    csv_writer.writerows(data_list)  # 데이터 추가
'''

- csv 구조 변경 (행열 변환)

In [ ]:
import csv

# 입력 파일과 출력 파일 이름\
input_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\cw_paperswithcode.csv'
output_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\paperswithcode.csv'
# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_filename, 'r', newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        data.append(row)

# 데이터를 풀어서 저장할 새로운 리스트 생성
unrolled_data = []
for row in data:
    method = row[0]
    category = row[1]
    models = row[2].split(', ')  # ,를 기준으로 model들을 분리
    for model in models:
        unrolled_data.append([method, category, model])

# 새로운 CSV 파일로 데이터를 쓰기
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in unrolled_data:
        csv_writer.writerow(row)

### 6. 모델 리스트 불러오기 (paperswithcode.csv)

In [ ]:
docs2_path = 'vscode/paperswithcode.csv'
docs2 = pd.read_csv(docs2_path)

In [ ]:
docs2.info() #model 2개 null

In [ ]:
docs2.describe()

In [ ]:
docs2

### 7. 모델 리스트를 패턴화하여 arxiv dataset에 적용

In [ ]:
tmp = docs2[docs2.method.str.contains('Computer Vision')]
model_list = tmp.iloc[:,2].tolist() #모델

model_counts = defaultdict(int) # key : value
patterns={}

# 패턴화 하기
for model in model_list:
    if isinstance(model,str): # model 의 객체가 string 이면
        patterns[model] = re.compile(re.escape(model),re.I)

# document filtering
filtered_cs_cv_docs = {model: cs_cv_docs[cs_cv_docs['abstract'].str.contains(pattern)] 
                      for model,pattern in patterns.items()}

for model,filtered_docs in filtered_cs_cv_docs.items():
    model_counts[model] = len(filtered_docs)

for model,count in model_counts.items():
    print(f"{model} : {count}")
    


In [ ]:
# 결론
# 생각했던 것에 비해 model_counts의 값이 작다.
# 정규표현식을 통한 대소문자 구분 및 문자열 구분 등을 확장하여 개선해본다.

### 8. 정규표현식을 이용하여 7번의 결과를 개선하기

In [ ]:
tmp="VGG"

# CASE1 -> GAN : 대소문자 구분 x & 문자열 구분 x -> legan, lovegan (0)
matching_rows1 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(f"{tmp}", re.I))| #정규표현식 re.I -> 대소문자 구별X
    cs_cv_docs['title'].str.contains(re.compile(f"{tmp}", re.I))
]
matching_count1 = len(matching_rows1)
print(matching_count1)


# CASE2 -> 대소문자 구분 o & 문자열 구분 o -> GAN(o) eGAN(x)

matching_rows2 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(rf'\b{tmp}\b')|
    cs_cv_docs['title'].str.contains(rf'\b{tmp}\b')
]
matching_count2 = len(matching_rows2)
print(matching_count2)


# CASE3 -> 대소문자 구분 x & 문자열 구분 o -> GAN & gan (o) -> 영진님 
   
matching_rows3 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b{tmp}\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b{tmp}\b',re.I))
]
matching_count3 = len(matching_rows3)
print(matching_count3)


#CASE4 -> 대소문자 구분 o & 문자열 구분x -> eGAN (o) egan(x)

matching_rows4 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(f"{tmp}")|
    cs_cv_docs['title'].str.contains(f"{tmp}")
]
matching_count4 = len(matching_rows4)
print(matching_count4)

In [ ]:
'''
<정리 CASE1-4>
1. 대소문자를 구분하지 않아야 함
    UNET,uNet,UNet

2. 문자열 구분은 애매함

2-1) 문자열 구분을 하면(2,3) 모델의 폭이 줄어들음 -> GAN) DCGAN, StyleGAN 불가
하지만 paperswithcode의 모델리스트가 어느정도 커버가 되긴 한다. 

2-2) 문자열 구분을 하지 않으면(1,4) 모델의 폭이 증가하나 CCT모델의 경우 CCTV 단어도 인식을 하기에 리스크가 크다.

'''

In [ ]:
# CASE5 -> 대소문자 구분 x & 문자열 구분 * -> 공백 및 숫자 인정 -> ResNet, ResNet 1 , ResNet1 

matching_rows5 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b\d*\s*{tmp}\s*\d*\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b\d*\s*{tmp}\s*\d*\b',re.I))
]
matching_count5 = len(matching_rows5)
print(matching_count5)


# CASE6 -> 대소문자 구분 x & 문자열 구분 * -> 공백 및 숫자 인정 + 's' 인정 -> GANs, VGGs 

matching_rows6 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b\d*\s*{tmp}s?\s*\d*\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b\d*\s*{tmp}s?\s*\d*\b',re.I))
]
matching_count6 = len(matching_rows6)
print(matching_count6)

In [ ]:
'''
<정리 CASE5-6>
1. 대소문자는 구분하지 않아야 함.
2. 문자열은 일부 구분
    공백 및 숫자는 허용하도록 함. -> VGG19
3. 모델+'s' 허용
    Abstract 분석 결과 GANs, VGGs 로 표기된 경우도 다수 있었음
'''

### 9. model_list (paperswithcode) + title & abstract (arxiv) 

In [ ]:
docs3.info()

In [ ]:
len(docs3[docs3['is_Reinforcement Learning']==True])

- 중복 칼럼 개수 확인

In [ ]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least2 = len(docs3[true_count >= 2])
true_count_least2

In [ ]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least3 = len(docs3[true_count >= 3])
true_count_least3

In [ ]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least4 = len(docs3[true_count >= 4])
true_count_least4

In [ ]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least5 = len(docs3[true_count >= 5])
true_count_least5

## test

In [ ]:
import pandas as pd
s1 = pd.Series(['house and parrot'])

tmp = 'parrot'
print(s1.str.contains(rf"\b{tmp}\b", case=False))
print(s1.str.match(rf'\b\d\s*{tmp_model}s?\s*\d*\b', case=False))

### IDEA (detail)
- 1) 중복되는 카테고리 처리는?
ex. Transformer 모델의 경우 CV인지 NLP인지 구분해줄 필요가 있을 것 같음.
- 2) journal-ref를 활용해볼까?

In [ ]:

tmp_docs=pd.read_csv('vscode/modified.csv')

tmp_docs.rename(columns={'is_Computer Vision': 'is_CV', 'is_Natural Language Processing': 'is_NLP',
                  'is_Reinforcement Learning':'is_RL','is_Audio':'is_Au','is_Sequential':'is_Se','is_Graphs':'is_Gr'}, inplace=True)

# 변경된 DataFrame을 CSV 파일로 저장
tmp_docs.to_csv('vscode/modified1.csv', index=False)  # index=False로 설정하여 인덱스 열을 저장하지 않도록 합니다.

In [ ]:
tmp_docs = pd.read_csv('vscode/modified1.csv')

In [ ]:
tmp_docs.info()

In [ ]:
true_count = tmp_docs[['is_CV','is_NLP','is_RL','is_Au','is_Se','is_Gr']].sum(axis=1)
true_count_least2 = len(tmp_docs[true_count >= 5])
true_count_least2

### categories(arxiv)와 method(paperswithcode) 간의 관계 분석

- CV -> cs.AI, cs.CV
- NLP -> cs.AI, cs.CL
- RL -> cs.LG
- Au -> cs.SD
- Se -> 
- Gr -> cs.DM


In [ ]:
categories= ['AI','AR','CC','CE','CG','CL','CR',
              'CV','CY','DB','DC','DL','DM','DS',
              'ET','FL','GL','GR','GT','HC','IR',
              'IT','LG','LO','MA','MM','MS','NA',
              'NE','NI','OH','OS','PF','PL','RO',
              'SC','SD','SE','SI','SY'] #40개

methods =['CV','NLP','RL','Au','Se','Gr']

result_dict={}
for method in methods:
    cnt_dict={}
    for category in categories:
        cnt_dict[category]=len(tmp_docs[(tmp_docs[f'is_{method}']==True) & (tmp_docs['categories'].str.contains(f'cs.{category}'))])
    result_dict[method] = sorted(cnt_dict.items(), key=lambda x: x[1],reverse=True) # value값 기준

In [ ]:
result_dict

In [ ]:
'''
<영진님 -> data_ai_fin2.csv>
Method: CV, Count: 37016
Category: CV, ratio: 57.77%
Category: LG, ratio: 46.67%
Category: AI, ratio: 19.21%
----------------------------
Method: NLP, Count: 67224
Category: LG, ratio: 45.31%
Category: CV, ratio: 36.47%
Category: CL, ratio: 26.18%
----------------------------
Method: RL, Count: 4603
Category: LG, ratio: 63.5%
Category: AI, ratio: 43.34%
Category: CV, ratio: 18.86%
----------------------------
Method: Au, Count: 257
Category: SD, ratio: 58.75%
Category: LG, ratio: 57.2%
Category: CL, ratio: 45.14%
----------------------------
Method: Se, Count: 4908
Category: LG, ratio: 47.78%
Category: CL, ratio: 47.05%
Category: CV, ratio: 22.43%
----------------------------
Method: Gr, Count: 15711
Category: CV, ratio: 46.1%
Category: LG, ratio: 43.42%
Category: AI, ratio: 24.52%
----------------------------

'''

In [ ]:
methods = ['CV','NLP','RL','Au','Se','Gr']
method_count = [len(tmp_docs[tmp_docs[f'is_{method}']==True]) for method in methods]              

i=0    
for method in methods:                
    top_categories = result_dict[method][:3] 
    print(f"Method: {method}, Count: {method_count[i]}")
    for category, category_count in top_categories:
        #print(f"Category: {category}, Count: {category_count}")
        print(f"Category: {category}, ratio: {round(category_count/method_count[i]*100,2)}%")
    i+=1
    print('----------------------------')

### CV와 NLP에서 중복되는 모델 찾기

In [ ]:
# method가 'a'인 행과 'b'인 행을 따로 추출합니다.
method_cv = docs[docs2['method'] == 'Computer Vision']
method_nlp = docs2[docs2['method'] == 'Natural Language Processing']

common_models = pd.merge(method_cv, method_nlp, on='model', how='inner') #model이 같은 행만 추출
display(common_models)

common_model_count = len(common_models)
display(common_model_count)

#### Abstract & Title 에 TNT,MHMA,GEE,ViLBERT,ClipBERT가 포함된 행의 개수 출력해보기 (정규표현식 유지)

In [ ]:
test_dict={}
models=['TNT','MHMA','GEE','ViLBERT','ClipBERT']
for model in tqdm(models):
    if isinstance(model, str):
        pattern = rf'\b\d*{model}s?\d*\b'
        tmp_df = docs1['abstract'].str.contains(pattern,case=False,regex=True) | docs1['title'].str.contains(pattern,case=False,regex=True) 
        test_dict[model]=len(tmp_df[tmp_df==True]) #tmp_df : pd.Series
test_dict

-> 이렇게 적은데,, 왜 NLP에서 cs.CV와 LG가 겹치냐고..왜